In [ ]:
# How the script works (Version 1.0):
# Contains a Class(AWS) and sub-Class(GCP):
# 1. Create both AWS and GCP buckets
# 2. Gets and multiple CSV files for upload into AWS bucket 
# 3. Upload to bucket and Unzip for data analytics
# 4. Import all csv files into Pandas df and merge as one large CSV file
# 5. Optionally Delete AWS bucket
# 6. Performance Analytics on whole file and output for further Analysis in GCP cloud
# 7. Now transfers/upload large CSV into GCP Bucket created for customer pick-up via REST API
# 8. Ingest csv Data into SQL DB for PowerBI feed
# 8. Optionally Delete GCP bucket but check if bucket exists in a list b4 deletion

# Next Target: 
# 1. download files on AWS & GCP via REST API
# 2. Perform more Cleansing and Data Minging



In [ ]:
# This works
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import urllib
import boto3
from boto3.session import Session
import shutil
import glob

from google.cloud import storage
import google.cloud.storage
import json
import os
import sys
import getpass

        
class CloudConnect:
    
    def __init__(self):
        self.create_zip_type = "zip"
        self.source_zip_file_format = ".zip"
        self.source_dir = "C:\\PythonTraining\\files\\Azurefiles\\mkt_data"
        self.source_zip_path_to_unpack = "C:\\PythonTraining\\files\\AWS\\zip_files\\market_data_zip"
        self.unpacked_dest_dir = "C:\\PythonTraining\\files\\AWS\\unpacked_files"
        self.zip_source_dir_path = "C:\\PythonTraining\\files\\AWS\\zip_files\\"
        self.zip_source_dir_filename = "market_data_zip.zip"
        self.filename = (self.zip_source_dir_path + self.zip_source_dir_filename)
        self.zip_dir_to_unpack = "C:\\PythonTraining\\files\\AWS\\zip_files\\"
        self.zip_filename_to_unpack ="market_data_zip"
        self.zip_fullpath_to_unpack = (self.zip_dir_to_unpack + self.zip_filename_to_unpack)
        self.concat_file_dir = "C:\\PythonTraining\\files\\Azurefiles\\mkt_data\\*.csv"
        self.read_dir_files = glob.glob(self.concat_file_dir)
        self.file_output_for_upload = "C:\\PythonTraining\\files\\AWS\\merged_for_upload\\combined_mkt_data.csv"
        self.merged_files = pd.concat([pd.read_csv(file) for file in self.read_dir_files], ignore_index=True)
        self.gcp_source_file_name = "C:\\PythonTraining\\files\\AWS\\merged_for_upload\\combined_mkt_data.csv"
#         self.df: DataFrame = pd.read_csv(self.gcp_source_file_name) #fix this later
    
#     Create aws bucket
    def create_aws_s3_bucket(self, aws_bucket_name: str):
    
        s3 = boto3.client("s3")
#         s3 = self.s3
        session = boto3.session.Session()
        current_region = session.region_name
        s3.create_bucket(
        Bucket=aws_bucket_name,
        CreateBucketConfiguration={"LocationConstraint": current_region}
        )

    
    def create_zip_files(self):
        return shutil.make_archive(self.dest_zip_name, self.create_zip_type, self.source_dir)
    
#     Upload zip files to AWS Bucket
    def upload_file_to_s3(self):
#     import boto3
        s3 = boto3.client('s3')
        filename = self.filename
        bucket_name = "olu-bucket1"
        return s3.upload_file(filename, bucket_name, filename)    
        
        
#     Unzip CSV files to work with
    def unpack_zip_files_in_aws(self):
        return shutil.unpack_archive((self.source_zip_path_to_unpack + self.source_zip_file_format), 
                                     self.source_zip_path_to_unpack)
                
#     Merge and Performance analysis on data
    def df_import_and_concat_csv(self):
        return self.merged_files.to_csv(self.file_output_for_upload)


    @staticmethod
    def delete_bucket_with_boto3(bucket_name: str):
        s3 = boto3.client("s3")
    
        s3.delete_bucket(
            Bucket=bucket_name
            )




# aws = CloudConnect("olu-bucket-for-data-upload12345")
# aws.create_aws_s3_bucket()
aws = CloudConnect()
# aws.create_aws_s3_bucket("olu-bucket1")
# aws.create_zip_files()
# aws.upload_file_to_s3()
# aws.unpack_zip_files_in_aws()
# aws.df_import_and_concat_csv()
# aws.delete_bucket_with_boto3("olu-bucket1")

#####################################################################
    
# send combined csv files to GCP
# 1. Create GCP bucket and upload combined files
# 2. Do some analytics
# 3. Download and Insert into SQL DB (IAAS Assumed)
# 4. Delete bucket


class ConnectGcpCloud(CloudConnect):
    
    def __init__(self):
        CloudConnect.__init__(self)
        
        self.key = getpass.getpass('enter key: ') #using getpass module to hide key
    
        self.PATH = os.path.join(os.getcwd(), self.key) 
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = self.PATH
        
    
#     create google bucket
    def create_gcp_bucket(self, gcp_bucket_name: str):
        """Create a new bucket in specific location with storage class"""
        storage_client = storage.Client()

        bucket = storage_client.bucket(gcp_bucket_name)
        bucket.storage_class = "COLDLINE"
        new_bucket = storage_client.create_bucket(bucket, location="europe-west2")

        print(
            "Created bucket {} in {} with storage class {}".format(
                new_bucket.name, new_bucket.location, new_bucket.storage_class
            )
        )
        return new_bucket

    
    def upload_file_to_gcp_blob(self,gcp_bucket_name,gcp_destination_blob_name):
        """Uploads a file to the bucket."""
        storage_client = storage.Client()
        bucket = storage_client.bucket(gcp_bucket_name)
        blob = bucket.blob(gcp_destination_blob_name)

        return blob.upload_from_filename(self.gcp_source_file_name)

        print(
            "File {} uploaded to {}.".format(
                self.gcp_source_file_name, gcp_destination_blob_name
            )
        )


#     Check if bucket Exists before Upload
    def list_gcp_buckets(self):
        
        """Lists all buckets."""
        storage_client = storage.Client()
        buckets = storage_client.list_buckets()

        file_lst = [bucket.name for bucket in buckets]
        return file_lst

    
    def delete_gcp_bucket(self, bucket_name):
        """Deletes a bucket. The bucket must be empty."""
        # bucket_name = "your-bucket-name"

        storage_client = storage.Client()

        bucket = storage_client.get_bucket(bucket_name)
        bucket.delete()

        print("Bucket {} deleted".format(bucket.name))

                


# Keep your credentials well (sample My First Project-d4d25cc66bfc.json)
gcp = ConnectGcpCloud()
# gcp.create_gcp_bucket("olu-bucket122")
# gcp.upload_file_to_gcp_blob("olu-bucket111", "gcp_blob_doc_144")
# gcp.list_gcp_buckets()
# gcp.delete_gcp_bucket("olu-bucket122")
# gcp.ingest_to_sql_server(df, "market_data") not needed again- new class created


In [ ]:
# Load data into sql table
# Need to integrate inot the larger class

import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import urllib

path = "C:\\PythonTraining\\files\\AWS\\merged_for_upload\\combined_mkt_data.csv"
df = pd.read_csv(path)

def ingest_to_sql_server(df: pd.DataFrame, table_name: str):
#     path = "C:\\PythonTraining\\files\\AWS\\merged_for_upload\\combined_mkt_data.csv"
#     df = pd.read_csv(path)

    params = urllib.parse.quote_plus(r'DRIVER={SQL+Server+Native+Client+11.0};SERVER=OLU;DATABASE=Python;Trusted_Connection=yes')
    conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
#print(conn_str)
    engine = create_engine(conn_str)
    df.to_sql(table_name, conn_str, index=False, if_exists='replace')
    
ingest_to_sql_server(df, 'market_data')